# CSV files for automatic_evaluation_comparison.ipynb

This note book contains the code that builds every csv files needed for the notebook 'automatic_evaluation_comparison'.

In [ ]:
import pandas as pd
import numpy as np

from glob import glob
from fractions import Fraction
from math import *

from chord_eval.metric import get_distance
from chord_eval.data_types import ChordType

Creation of a csv file corpus.csv for each sub folder of the chord-eval folder with the average distance between both annotations of every pieces in the folder.

In [ ]:
for df_path in glob("../Data/chord-eval/*", recursive=True):
    parameter = df_path.split('/')[3]
    
    path_list = []
    avg_bin_acc_list = []
    avg_root_acc_list = []
    avg_triad_acc_list = []
    avg_7th_acc_list = []
    avg_inv_acc_list = []
    avg_sps_list = []
    avg_tbt_list = []
    avg_vl_list = []

    for df_path in glob("../Data/chord-eval/"+parameter+"/*_resultschord-eval.tsv", recursive=True):



        results_df = pd.read_csv(df_path, sep='\t', index_col=0, converters={'duration': Fraction})

        results_df['gt_chord_type'] = results_df['gt_chord_type'].apply(lambda r : ChordType[r.split(".")[1]])
        results_df['est_chord_type'] = results_df['est_chord_type'].apply(lambda r : ChordType[r.split(".")[1]])

        results_df['sps_distance'] = results_df.apply(lambda r : get_distance(distance='SPS',
                                                                              root1=r.gt_chord_root,
                                                                              root2=r.est_chord_root,
                                                                              chord_type1=r.gt_chord_type,
                                                                              chord_type2=r.est_chord_type,
                                                                              inversion1=r.gt_chord_inv,
                                                                              inversion2=r.est_chord_inv
                                                                              ), axis=1)

        results_df['tbt_distance'] = results_df.apply(lambda r : get_distance(distance='tone by tone',
                                                                              root1=r.gt_chord_root,
                                                                              root2=r.est_chord_root,
                                                                              chord_type1=r.gt_chord_type,
                                                                              chord_type2=r.est_chord_type,
                                                                              inversion1=r.gt_chord_inv,
                                                                              inversion2=r.est_chord_inv,
                                                                              bass_weight=3,
                                                                              root_weight=3
                                                                              ), axis=1)

        results_df['vl_distance'] = results_df.apply(lambda r : get_distance(distance='voice leading',
                                                                              root1=r.gt_chord_root,
                                                                              root2=r.est_chord_root,
                                                                              chord_type1=r.gt_chord_type,
                                                                              chord_type2=r.est_chord_type,
                                                                              inversion1=r.gt_chord_inv,
                                                                              inversion2=r.est_chord_inv,
                                                                              bass_weight=3
                                                                              ), axis=1)

        results_df['vl_distance'] = results_df['vl_distance'].apply(lambda r :
                                                                    r/max(results_df['vl_distance']))

        path_list.append(df_path)
        avg_bin_acc_list.append(float(np.average(results_df['full_correct'], weights=results_df['duration'])))
        avg_root_acc_list.append(float(np.average(results_df['root_correct'], weights=results_df['duration'])))
        avg_triad_acc_list.append(float(np.average(results_df['triad_correct'], weights=results_df['duration'])))
        avg_7th_acc_list.append(float(np.average(results_df['7th_correct'], weights=results_df['duration'])))
        avg_inv_acc_list.append(float(np.average(results_df['inv_correct'], weights=results_df['duration'])))
        avg_sps_list.append(float(np.average(results_df['sps_distance'], weights=results_df['duration'])))
        avg_tbt_list.append(float(np.average(results_df['tbt_distance'], weights=results_df['duration'])))
        avg_vl_list.append(float(np.average(results_df['vl_distance'], weights=results_df['duration'])))
    
    Chord_symbol_recall_df = pd.DataFrame({'path': path_list,
                                           'binary_accuracy' : avg_bin_acc_list,
                                           'root_accuracy' : avg_root_acc_list,
                                           'triad_accuracy' : avg_triad_acc_list,
                                           '7th_accuracy' : avg_7th_acc_list,
                                           'inversion_accuracy' : avg_inv_acc_list,
                                           'average_sps' : avg_sps_list,
                                           'average_tbt' : avg_tbt_list,
                                           'average_vl' : avg_vl_list
                                           })

    Chord_symbol_recall_df['name'] = Chord_symbol_recall_df.path.apply(lambda r :
                                        r[len('../Data/chord-eval/'+parameter+'/'):len(r)-len('_resultschord-eval.tsv')])

    Chord_symbol_recall_df.to_csv('../Data/chord-eval/'+parameter+'/corpus.csv', index=False)